## GenerativeAI4DS-II
## Lab 2. Introduction to LangChain


### In this lab you will learn how to compose basic prompts and have these prompts executed on OpenAI using LangChain.Furthermore you will learn how to format the output for subsequent processing.

#### LangChain saves time and development effort by providing an interface to interact with Large Language Models (LLMs).


### Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [31]:
OPENAI_API_KEY="YOU-NEED-YOUR-OWN-KEY"

In [2]:
!pip install --upgrade langchain;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.6 MB/s eta 0:00:00


In [3]:

!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [6]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.1 MB/s eta 0:00:00


In [4]:
import os
import openai



### 1. Model

In [7]:
from langchain.chat_models import ChatOpenAI

In [24]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model='gpt-3.5-turbo',openai_api_key=OPENAI_API_KEY)
chat

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7cc39d14fac0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7cc39d1a9210>, temperature=0.0, openai_api_key='sk-proj-K0uwYVJs2UMN6cUaS2BsT3BlbkFJ166vgXlfK7OEMyMfDN8j', openai_proxy='')

### 2. Prompt template

In [8]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

### 3. Output Template

In [9]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [10]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased as a gift for someone else? Answer True if yes,False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days did it take for the product to arrive? If this information is not found,output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any sentences about the value or price, and output them as a comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [11]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [12]:
format_instructions = output_parser.get_format_instructions()

In [13]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes,False if not or unknown.\n\t"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found,output -1.\n\t"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.\n}\n```'

### 3. Prompt Definition
#### Using the model and the templates previously defined we want to process a customer review and extract some information, namely: (1) was it purchased as a gift ?, (2) delivery days and (3) price value.
#### Moreover we want to process the output to have it formatted as a JSON.

In [14]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado.
It arrived in two days, just in time for my wife's anniversary present.I think my wife liked it so much she was speechless.
So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn.
It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features. """

In [17]:
from langchain.prompts.chat import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template=review_template_2)

In [18]:
prompt

ChatPromptTemplate(input_variables=['format_instructions', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: {text}\n\n{format_instructions}\n'))])

In [19]:
messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

In [20]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings: candle blower, gentle breeze, windy city, and tornado.
It arrived in two days, just in time for my wife's anniversary present.I think my wife liked it so much she was speechless.
So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn.
It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features. 

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

### 4. Prompt Execution

In [25]:
response = chat(messages)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [26]:
print(response.content)

```json
{
	"gift": false,
	"delivery_days": 2,
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [27]:
output_dict = output_parser.parse(response.content)

In [28]:
type(output_dict)

dict

In [29]:
output_dict

{'gift': False,
 'delivery_days': 2,
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [30]:
output_dict.get('delivery_days')

2